# Multi objective optimization post-processing analysis

In [ ]:
#import WarmupFUSE
using FUSE
using Plots

In [ ]:
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_noonetime"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_slvv"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_neo"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_neo_SMSEMOA"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_neo_SPEA2"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_neo_SPEA2_HCD"
path= "optimization_run_LTSorHTS_ohtf"
path= "optimization_run_LTSorHTS_ohtf_CCMO"
path= "optimization_run_LTSorHTS_ohtf_fixHFS2"
path= "optimization_run_LTSorHTS_ohtf_fixHFS_req"
path= "optimization_run_LTSorHTS_ohtf_fixHFS_req_dens"
path= "optimization_run_LTSorHTS_ohtf_fixHFS_req_dens_flattop"
path = "optimization_run_LTSorHTS_ohtf_fixHFS_req_dens_flattop_fixHFSagain"

In [ ]:
IMAS.update_ExtractFunctionsLibrary!()
# use this to regenerate CSV cache file
all_dirs = filter(isdir,sort(readdir(path; join=true)))
println(length(all_dirs))
dirs = filter(x->!isfile(joinpath(x,"error.txt")) && isfile(joinpath(x,"dd.json")),all_dirs)#[1:100]
println(length(dirs))
outputs=FUSE.extract(dirs, filter_invalid=:cols);

# cache extrated information to CSV file
import DelimitedFiles
DelimitedFiles.writedlm(joinpath(path,"extract.csv"), Iterators.flatten(([names(outputs)], eachrow(outputs))), ',')

In [ ]:
# error analysis
errors=FUSE.categorize_errors(all_dirs;do_plot=true,show_first_line=true)
#println(read("optimization_run_LTSorHTS_ohtf/2023-04-28T11:34:32.300__3823851/error.txt",String))

In [ ]:
# use this to load CSV cache file
outputs=FUSE.DataFrames.DataFrame(FUSE.CSV.File(joinpath(path,"extract.csv")));
println(length(outputs[:,1]))

In [ ]:
# this is just to list the fields that can be queried
IMAS.ExtractFunctionsLibrary

In [ ]:
N=length(outputs[:,"Pelectric_net"])
X=LinRange(1,N,50)

histogram2d(outputs[:,"Pelectric_net"],bins=(X,LinRange(0,500,50)),ylabel="Pelectric net", xlabel="Indiviuals")
#scatter!(outputs[:,"Pelectric_net"],ylim=(-500,500))
display(hline!([200.0], ls=:dash, label=""))

histogram2d(outputs[:,"ip_ohm"],bins=(X,LinRange(-5,5,50)),ylabel="Ip ohmic", xlabel="Indiviuals")
#scatter!(outputs[:,"ip_ohm"],ylim=(-50,50))
display(hline!([0.0], ls=:dash, label=""))

cname="HTS"; c=(outputs[:,"TF_material"].=="ReBCO").+(outputs[:,"OH_material"].=="ReBCO").*2.0;
histogram2d(c,bins=(X,LinRange(-5,5,50)),ylabel=cname, xlabel="Indiviuals")

In [ ]:
import Statistics
for name in names(outputs)
    if !(typeof(outputs[1,name]) <: Number) || all(isnan.(outputs[:,name]))
        continue
    end
    y=outputs[:,name]
    y_nonan=y[@. !isnan.(y)]
    m=Statistics.median(y_nonan)
    σ=Statistics.median(Statistics.median(abs.(y_nonan.-m)))*10
    if σ != 0.0
        display(histogram2d(y,bins=(X,LinRange(max(m-σ,minimum(y_nonan)),min(m+σ,maximum(y_nonan)),100)),ylabel=name))
    end
end

In [ ]:
# x axis
xname="capital_cost"; x=outputs[:,xname]
#xname="βpol"; x=outputs[:,xname]
#xname="Pec [MW]"; x=outputs[:,"Pec"]

# y axis
yname="βn"; y=outputs[:,yname]
#yname="ip_bs/ip_bs_aux_ohm"; y=outputs[:,"ip_bs"]./outputs[:,"ip_bs_aux_ohm"];
#yname="Pec=R0*ne*ip_aux(5+zeff)/(Te*0.09)"; y= @. outputs[:,"<ne>"]/1E20*outputs[:,"R0"]*outputs[:,"ip_aux"]*(5.0+outputs[:,"<zeff>"])/(0.09*outputs[:,"<Te>"])

# z axis
#zname="R0"; z=outputs[:,zname]

# color
#cname="individual"; c=1:length(outputs[:,xname]);clim=(1,Inf);
#cname="log10(flattop)"; c=log10.(outputs[:,"flattop"]);clim=(-Inf,Inf);
#cname="Pelectric_net"; c=outputs[:,cname];clim=(-Inf,Inf);
#cname="Ip aux [MA]"; c=outputs[:,"ip_aux"];clim=(-Inf,Inf);
#cname="fGW"; c=outputs[:,"fGW"];clim=(-Inf,Inf);
#cname="<zeff>"; c=outputs[:,"<zeff>"];clim=(-Inf,Inf);
#cname="R0 [m]"; c=outputs[:,"R0"];clim=(-Inf,Inf);
cname="B0 [T]"; c=outputs[:,"B0"];clim=(-Inf,Inf);
#cname="ip_ohm"; c=outputs[:,cname];clim=(-Inf,Inf);
#cname="ip"; c=outputs[:,cname];clim=(-Inf,Inf);
#cname="Pec [MW]"; c=outputs[:,"Pec"];clim=(-Inf,Inf);
#cname="ip_bs_aux_ohm/ip"; c=outputs[:,"ip_bs_aux_ohm"]./outputs[:,"ip"];clim=(-Inf,Inf);
#cname="ip_ohm"; c=abs.(outputs[:,"ip_ohm"]);clim=(-Inf,Inf);
#cname="ip_bs_aux_ohm/ip"; c=outputs[:,"ip_bs_aux_ohm"]./outputs[:,"ip"];clim=(-Inf,Inf);
#cname="Ip aux [MA]"; c=outputs[:,"ip_aux"];clim=(-Inf,Inf);

cname="HTS"; c=(outputs[:,"TF_material"].=="ReBCO").+(outputs[:,"OH_material"].=="ReBCO").*2.0; clim=(0,4)

# selection
sname0="Pelectric_net"; s0=outputs[:,sname0]
sname1="ip_ohm"; s1=outputs[:,sname1]
sname2="flattop"; s2=outputs[:,sname2]

# subselection criterion
Δf=10
Δohm=1000
flattop=11.5
n=length(x)
index=1:n
index=findall(  abs.(s0.-200).<Δf .&& (abs.(s1).<Δohm) .&& (s2.>flattop) .&& (index.> 0000))
#index=findall( abs.(s0.-200).<Δf .&& (index.>0000))
#index=findall( abs.(s0.-200).<Δf)
#index=findall( (index.>n-1000))
annot=map(x->(x, :center, 3, "courier"), index)

# plotting
println("$(length(index)) points")
scatter(x[index], y[index], marker_z=c[index] ,xlabel=xname, ylabel=yname, colorbar_title=cname, marker=:circle,
    markersize=5, markerstrokewidth=0, label="", clim=clim, alpha=0.5,
#    series_annotations=annot,
#    aspect_ratio=:equal,
    xlim=(0,20), ylim=(0,Inf))
#    xlim=(0,10), ylim=(0,4.5))
#    xlim=(0,10), ylim=(0,5))

pindex=index[FUSE.pareto_front([[x[index[k]],y[index[k]]] for k in 1:length(index)])]
sort!(pindex,by=i->y[i])
sort!(pindex,by=i->x[i])
println(pindex)
pannot=map(x->("\n$x", :right, 3, "courier", :red), pindex)
plot!(x[pindex],y[pindex],series_annotations=pannot,color=:blue, label="pareto front")

In [ ]:
n=3946
dir=outputs[n,"dir"]
#n=length(dirs)
println(dir)
dd,ini,act=FUSE.load(dir)
#FUSE.ActorHFSsizing(dd,act;verbose=true)
FUSE.digest(dd)
#ini.requirements